In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/facebook-combined/facebook_combined.txt
/kaggle/input/facebook-ego/698.featnames
/kaggle/input/facebook-ego/3980.featnames
/kaggle/input/facebook-ego/686.egofeat
/kaggle/input/facebook-ego/3437.circles
/kaggle/input/facebook-ego/1912.featnames
/kaggle/input/facebook-ego/1684.featnames
/kaggle/input/facebook-ego/1684.egofeat
/kaggle/input/facebook-ego/3437.feat
/kaggle/input/facebook-ego/3437.featnames
/kaggle/input/facebook-ego/686.circles
/kaggle/input/facebook-ego/107.featnames
/kaggle/input/facebook-ego/348.egofeat
/kaggle/input/facebook-ego/0.feat
/kaggle/input/facebook-ego/698.feat
/kaggle/input/facebook-ego/686.feat
/kaggle/input/facebook-ego/107.feat
/kaggle/input/facebook-ego/348.feat
/kaggle/input/facebook-ego/698.circles
/kaggle/input/facebook-ego/0.circles
/kaggle/input/facebook-ego/414.edges
/kaggle/input/facebook-ego/414.feat
/kaggle/input/facebook-ego/1684.feat
/kaggle/input/facebook-ego/698.egofeat
/kaggle/input/facebook-ego/414.circles
/kaggle/input/facebo

In [62]:
# ===== CONFIG =====
EDGES_PATH  = "/kaggle/input/facebook-combined/facebook_combined.txt"
LABELS_PATH = None   # có file nhãn thật (u,v,label) thì điền path; không có để None

NODE_TARGET   = 414
TOPK          = 20
POS_FRAC_HIDE = 0.10   # khi tự tạo nhãn: ẩn 10% cạnh thật làm positive
NEG_RATIO     = 1.0    # số negative ≈ số positive
SEED          = 42

# ===== IMPORTS =====
import os, math, gc
import numpy as np, pandas as pd, networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

np.random.seed(SEED)


In [63]:
def load_graph(path: str) -> nx.Graph:
    G = nx.Graph()
    with open(path, "r") as f:
        for line in f:
            u, v = map(int, line.split())
            G.add_edge(u, v)
    return G

G = load_graph(EDGES_PATH)
print("Nodes:", G.number_of_nodes(), "| Edges:", G.number_of_edges(), "| Directed?", G.is_directed())


Nodes: 4039 | Edges: 88234 | Directed? False


In [64]:
def features_for_pairs(G: nx.Graph, pairs):
    """Trả về DataFrame đặc trưng cho list pairs=[(u,v),...]"""
    pairs = [(int(u), int(v)) for (u,v) in pairs if u in G and v in G and u!=v]

    # 1) Common Neighbors
    def cn_count(u, v):
        try:    return sum(1 for _ in nx.common_neighbors(G, u, v))
        except: return 0
    cn = {(u,v): cn_count(u,v) for (u,v) in pairs}

    # 2) Jaccard
    jacc = {(u,v): s for (u,v,s) in nx.jaccard_coefficient(G, pairs)}

    # 3) Adamic–Adar
    aa = {(u,v): s for (u,v,s) in nx.adamic_adar_index(G, pairs)}

    # 4) Preferential Attachment
    pa = {(u,v): s for (u,v,s) in nx.preferential_attachment(G, pairs)}

    # 5) Shortest Path Length (không có đường đi -> 99)
    spl = {}
    for (u,v) in pairs:
        try: spl[(u,v)] = nx.shortest_path_length(G, u, v)
        except: spl[(u,v)] = 99

    rows = []
    for (u,v) in pairs:
        rows.append({
            "u": u, "v": v,
            "cn": cn.get((u,v), 0),
            "jaccard": jacc.get((u,v), 0.0),
            "adamic_adar": aa.get((u,v), 0.0),
            "pref_attach": pa.get((u,v), 0),
            "sp_len": spl.get((u,v), 99),
        })
    return pd.DataFrame(rows)


In [65]:
def build_supervised_dataset(G: nx.Graph, labels_path: str = None,
                             pos_frac_hide=0.1, neg_ratio=1.0, seed=42):
    """
    - Có labels.csv (u,v,label) -> dùng trực tiếp, trích đặc trưng trên G hiện tại.
    - Không có -> ẩn một phần edges làm positive, sample non-edges làm negative,
      trích đặc trưng trên G đã bỏ positive để tránh rò rỉ.
    Trả về (X, y, G_feat) với G_feat là đồ thị dùng trích feature.
    """
    rng = np.random.default_rng(seed)

    if labels_path and os.path.exists(labels_path):
        df_lab = pd.read_csv(labels_path)
        assert set(["u","v","label"]).issubset(df_lab.columns), "labels.csv cần cột u,v,label"
        pairs = list(zip(df_lab["u"].astype(int), df_lab["v"].astype(int)))
        X = features_for_pairs(G, pairs)
        X["label"] = df_lab["label"].astype(int).values
        return X.drop(columns=["u","v"]), X["label"], G

    # --- Tự tạo nhãn ---
    edges = list(G.edges()); rng.shuffle(edges)
    m_hide = max(1, int(pos_frac_hide * len(edges)))
    pos_edges = edges[:m_hide]

    G_feat = G.copy()
    G_feat.remove_edges_from(pos_edges)  # tránh leakage

    # Negative = non-edges (cùng số lượng positive)
    target_neg = int(neg_ratio * m_hide)
    neg_edges, nodes = set(), list(G_feat.nodes())
    while len(neg_edges) < target_neg:
        u, v = rng.choice(nodes, 2, replace=False)
        if not G_feat.has_edge(u, v):
            neg_edges.add((int(u), int(v)))
    neg_edges = list(neg_edges)

    df_pos = features_for_pairs(G_feat, pos_edges); df_pos["label"] = 1
    df_neg = features_for_pairs(G_feat, neg_edges); df_neg["label"] = 0
    df_all = pd.concat([df_pos, df_neg], ignore_index=True)

    X = df_all.drop(columns=["u","v","label"])
    y = df_all["label"].astype(int)
    return X, y, G_feat

X, y, G_feat_graph = build_supervised_dataset(
    G, LABELS_PATH, pos_frac_hide=POS_FRAC_HIDE, neg_ratio=NEG_RATIO, seed=SEED
)
print("Supervised dataset:", X.shape, "| positives:", int(y.sum()), "| negatives:", int((1-y).sum()))


Supervised dataset: (17646, 5) | positives: 8823 | negatives: 8823


In [66]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

rf = RandomForestClassifier(n_estimators=300, n_jobs=-1, random_state=SEED)
rf.fit(X_train, y_train)

proba = rf.predict_proba(X_test)[:,1]
pred  = (proba >= 0.5).astype(int)

print("ROC-AUC:", roc_auc_score(y_test, proba).round(4))
print(classification_report(y_test, pred, digits=4))

# (tuỳ chọn) xem tầm quan trọng đặc trưng
imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
display(imp)


ROC-AUC: 0.9902
              precision    recall  f1-score   support

           0     0.9749    0.9677    0.9713      1765
           1     0.9679    0.9751    0.9715      1765

    accuracy                         0.9714      3530
   macro avg     0.9714    0.9714    0.9714      3530
weighted avg     0.9714    0.9714    0.9714      3530



adamic_adar    0.429852
cn             0.264146
jaccard        0.206285
sp_len         0.067603
pref_attach    0.032114
dtype: float64

In [67]:
def candidates_by_distance(G: nx.Graph, u0: int, exact_dist=None, min_dist=None):
    """Ứng viên KHÔNG kết nối trực tiếp với u0, có đường đi và thoả khoảng cách."""
    nbrs = set(G.neighbors(u0))
    out = []
    # nhanh hơn: quét distances gần với single_source
    dists = nx.single_source_shortest_path_length(G, u0, cutoff=6)
    for v, d in dists.items():
        if v == u0 or v in nbrs: 
            continue
        if exact_dist is not None and d == exact_dist:
            out.append(v)
        elif min_dist is not None and d >= min_dist:
            out.append(v)
    return out

def recommend_for_candidates(u0: int, candidates, model, G_feat, topk=10):
    pairs = [(u0, v) for v in candidates]
    feats = features_for_pairs(G_feat, pairs)  # trích trên G_feat (đã tránh rò rỉ nếu tự tạo nhãn)
    probs = model.predict_proba(feats[["cn","jaccard","adamic_adar","pref_attach","sp_len"]])[:,1]
    feats["p"] = probs
    return feats.sort_values("p", ascending=False).head(topk).reset_index(drop=True)

u0 = NODE_TARGET

# Nhóm 1: dist=2 (bạn của bạn)
cand_d2 = candidates_by_distance(G, u0, exact_dist=2)
top_d2  = recommend_for_candidates(u0, cand_d2, rf, G_feat_graph, topk=TOPK)

# Nhóm 2: dist>=3 (bạn của bạn của bạn…)
cand_d3p = candidates_by_distance(G, u0, min_dist=3)
top_d3p  = recommend_for_candidates(u0, cand_d3p, rf, G_feat_graph, topk=TOPK)

print(f"#candidates dist=2 : {len(cand_d2)}")
print(f"#candidates dist≥3: {len(cand_d3p)}")

print("\nTop-10 (dist=2):")
for i, r in top_d2.iterrows():
    print(f"{i+1:>2}. v={int(r.v):<5} p={r.p:.3f}  CN={int(r.cn)}  J={r.jaccard:.4f}  AA={r.adamic_adar:.3f}  dist={int(r.sp_len)}")

print("\nTop-10 (dist≥3):")
for i, r in top_d3p.iterrows():
    print(f"{i+1:>2}. v={int(r.v):<5} p={r.p:.3f}  CN={int(r.cn)}  J={r.jaccard:.4f}  AA={r.adamic_adar:.3f}  dist={int(r.sp_len)}")


#candidates dist=2 : 1217
#candidates dist≥3: 2662

Top-10 (dist=2):
 1. v=545   p=0.997  CN=14  J=0.0828  AA=3.336  dist=2
 2. v=526   p=0.997  CN=11  J=0.0498  AA=2.448  dist=2
 3. v=458   p=0.997  CN=10  J=0.0629  AA=2.384  dist=2
 4. v=366   p=0.997  CN=28  J=0.1379  AA=6.619  dist=2
 5. v=527   p=0.997  CN=25  J=0.1562  AA=5.839  dist=2
 6. v=397   p=0.997  CN=11  J=0.0625  AA=2.487  dist=2
 7. v=402   p=0.997  CN=27  J=0.1598  AA=6.425  dist=2
 8. v=495   p=0.993  CN=10  J=0.0641  AA=2.483  dist=2
 9. v=479   p=0.993  CN=12  J=0.0723  AA=2.787  dist=2
10. v=493   p=0.993  CN=13  J=0.0734  AA=2.966  dist=2
11. v=404   p=0.993  CN=15  J=0.0867  AA=3.533  dist=2
12. v=417   p=0.990  CN=27  J=0.1561  AA=6.351  dist=2
13. v=537   p=0.990  CN=14  J=0.0854  AA=3.215  dist=2
14. v=444   p=0.990  CN=16  J=0.0925  AA=3.700  dist=2
15. v=387   p=0.990  CN=16  J=0.1013  AA=3.716  dist=2
16. v=487   p=0.987  CN=16  J=0.1000  AA=3.666  dist=2
17. v=482   p=0.987  CN=16  J=0.1000  AA=3.667  dis

In [68]:
out_d2  = "/kaggle/working/recommendations_414_dist2.csv"
out_d3p = "/kaggle/working/recommendations_414_dist_ge3.csv"
top_d2.to_csv(out_d2, index=False)
top_d3p.to_csv(out_d3p, index=False)
print("Saved:", out_d2, "and", out_d3p)

# Bảng thống kê nhanh để so sánh p
def quick_stats(df, tag):
    return {
        "group": tag, "count": len(df),
        "p_mean": float(df["p"].mean()),
        "p_median": float(df["p"].median()),
        "p_min": float(df["p"].min()),
        "p_max": float(df["p"].max()),
    }

stats = pd.DataFrame([quick_stats(top_d2, "dist=2"),
                      quick_stats(top_d3p, "dist≥3")])
display(stats)


Saved: /kaggle/working/recommendations_414_dist2.csv and /kaggle/working/recommendations_414_dist_ge3.csv


,group,count,p_mean,p_median,p_min,p_max
0,dist=2,20,0.992167,0.993333,0.986667,0.996667
1,dist≥3,20,0.069298,0.016667,0.016667,0.227193
